In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

import seaborn as sns
sns.set(style="ticks")
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
import xgboost as xgb
import matplotlib.pyplot as plt
from numpy import median
pd.options.display.max_rows = 200
# Any results you write to the current directory are saved as output.

In [ ]:
train = pd.read_csv("../input/train.csv")
test = pd.read_csv("../input/test.csv")

In [ ]:
train_copy = train.copy() 

In [ ]:
train.info()

In [ ]:
#pd.qcut(train.Age,q= [0, .25, .5, .75, 1.] ,labels=False)

In [ ]:
_ = sns.distplot(train.Age.dropna())


In [ ]:
from sklearn import preprocessing
_ = sns.distplot(preprocessing.StandardScaler().fit_transform(train[['Age']].dropna()))


In [ ]:
_ = sns.distplot(preprocessing.MinMaxScaler().fit_transform(train[['Age']].dropna()))

In [ ]:
_ = sns.distplot(preprocessing.QuantileTransformer().fit_transform(train[['Age']].dropna()))

In [ ]:
_ = sns.distplot(preprocessing.Normalizer().fit_transform(train[['Age','Fare']].dropna())[:,0])

In [ ]:
_ = sns.distplot(preprocessing.Normalizer().fit_transform(train[['Age','Fare']].dropna())[:,1])

In [ ]:
train[['Age','Fare']].head()

In [ ]:
preprocessing.Normalizer().fit_transform(train[['Age','Fare']].dropna())[:,0].var()

In [ ]:
preprocessing.Normalizer().fit_transform(train[['Age','Fare']].dropna())[:,1].var()

In [ ]:
preprocessing.Normalizer().fit_transform(train[['Age','Fare']].dropna())

In [ ]:
pd.DataFrame(preprocessing.Normalizer().fit_transform(train[['Age','Fare']].dropna()),columns=['Age', 'Fare']).plot.kde()

In [ ]:
pd.DataFrame(preprocessing.MinMaxScaler().fit_transform(train[['Age','Fare']].dropna()),columns=['Age', 'Fare']).plot.kde()

In [ ]:
pd.DataFrame(preprocessing.StandardScaler().fit_transform(train[['Age','Fare']].dropna()),columns=['Age', 'Fare']).plot.kde()

In [ ]:
pd.DataFrame(preprocessing.StandardScaler().fit_transform(test[['Age','Fare']].dropna()),columns=['Age', 'Fare']).plot.kde()

In [ ]:
#_ = sns.distplot(preprocessing.Normalizer().fit_transform(train[['Age']].dropna()))

In [ ]:
preprocessing.KBinsDiscretizer(n_bins=[2,2,2,2], encode='ordinal').fit_transform(train.loc[:,['Age','Fare','SibSp','Parch']].dropna())

In [ ]:
train.Embarked.value_counts()

In [ ]:
train.loc[train.Embarked.isnull(),'Embarked']='Q'

In [ ]:
train['IsCabinNull'] =  train.Cabin.isnull().astype(int)
test['IsCabinNull'] =  test.Cabin.isnull().astype(int)

In [ ]:
import copy
train_objs_num = len(train)
dataset = pd.concat(objs=[train, test],sort=False)
dataset['ngroup'] = dataset.groupby(['Pclass','Sex']).ngroup()
dataset = dataset['ngroup']
train['ngroup'] = copy.copy(dataset[:train_objs_num])
test['ngroup'] = copy.copy(dataset[train_objs_num:])

In [ ]:
train[['Name', 'Age']].iloc[10:20, :]

In [ ]:
print(train.Age.median())
print(test.Age.median())

In [ ]:
train.loc[train.Age.isnull(), 'Age'] = 28
test.loc[test.Age.isnull(), 'Age'] = 28

In [ ]:
print((train.Fare == 0).sum())
print((test.Fare == 0).sum())

In [ ]:
print(train.Fare.median())
print(test.Fare.median())

In [ ]:
train.loc[(train.Fare==0), 'Fare'] = train.Fare.median()
test.loc[(test.Fare==0), 'Fare'] = test.Fare.median()
test.loc[(test.Fare.isnull()), 'Fare'] = test.Fare.median()

In [ ]:
train['age_log'] = np.log(train['Age'])
test['age_log'] = np.log(test['Age'])
train['fare_log'] = np.log(train['Fare'])
test['fare_log'] = np.log(test['Fare'])

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(train[['Age']])
train['age_scale'] = scaler.transform(train[['Age']])
test['age_scale'] = scaler.transform(test[['Age']])
scaler = StandardScaler()
scaler.fit(train[['Fare']])
train['fare_scale'] = scaler.transform(train[['Fare']])
test['fare_scale'] = scaler.transform(test[['Fare']])

In [ ]:
#https://www.kaggle.com/ldfreeman3/a-data-science-framework-to-achieve-99-accuracy
train['SibParchSum'] = train['SibSp'] + train['Parch']
test['SibParchSum'] = test['SibSp'] + test['Parch']
train['IsWithFamily'] = (train['SibParchSum']>0).astype(int)
test['IsWithFamily'] = (test['SibParchSum']>0).astype(int)
from sklearn.preprocessing import KBinsDiscretizer
binner = KBinsDiscretizer(encode='ordinal')
binner.fit(train[['Fare']])
train['Fare_bins'] = binner.transform(train[['Fare']])
test['Fare_bins'] = binner.transform(test[['Fare']])
binner = KBinsDiscretizer(encode='ordinal')
binner.fit(train[['Age']])
train['age_bins'] = binner.transform(train[['Age']])
test['age_bins'] = binner.transform(test[['Age']])

In [ ]:
from sklearn.preprocessing import OrdinalEncoder
encoder = OrdinalEncoder()
encoder.fit(train[['Sex']])
train['sex_integer'] = encoder.transform(train[['Sex']])
test['sex_integer'] = encoder.transform(test[['Sex']])
encoder = OrdinalEncoder()
encoder.fit(train[['Embarked']])
train['embarked_integer'] = encoder.transform(train[['Embarked']])
test['embarked_integer'] = encoder.transform(test[['Embarked']])

In [ ]:
np.logspace(-2, 1, 10)

In [ ]:
np.logspace(-2, -1, 3)

In [ ]:
train_after_normalizer = preprocessing.Normalizer().fit_transform(train[['Age', 'Fare','SibSp', 'Parch','sex_integer','embarked_integer','Pclass']])

In [ ]:
train_after_normalizer = pd.DataFrame(train_after_normalizer, columns = ['Age', 'Fare','SibSp', 'Parch','sex_integer','embarked_integer','Pclass'])

In [ ]:
#https://scikit-learn.org/stable/auto_examples/preprocessing/plot_discretization_classification.html#sphx-glr-auto-examples-preprocessing-plot-discretization-classification-py
#https://stackoverflow.com/questions/34165731/a-column-vector-y-was-passed-when-a-1d-array-was-expected
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import KBinsDiscretizer, PolynomialFeatures
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import GradientBoostingClassifier, AdaBoostClassifier
from sklearn.utils.testing import ignore_warnings
from sklearn.exceptions import ConvergenceWarning

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import  OrdinalEncoder

from sklearn.metrics import roc_auc_score, log_loss, f1_score

print(__doc__)

h = .02  # step size in the mesh


def get_name(estimator):
    name = estimator.__class__.__name__
    if name == 'Pipeline':
        name = [get_name(est[1]) for est in estimator.steps]
        name = ' + '.join(name)
    return name

# numeric_features = ['Age', 'Fare']
# numeric_transformer = Pipeline(steps=[
#     ('imputer', SimpleImputer(strategy='median')),
#     ('scaler', StandardScaler())])

# categorical_features = ['Embarked', 'Sex']
# categorical_transformer = Pipeline(steps=[
#    # ('imputer', SimpleImputer(strategy='constant', fill_value='most_frequent')),
#     ('onehot', OrdinalEncoder())])
# #handle_unknown='ignore'
# preprocessor = ColumnTransformer(
#     transformers=[
#     #    ('num', numeric_transformer, numeric_features),
#         ('cat', categorical_transformer, categorical_features)], remainder='passthrough')


# clf = Pipeline(steps=[('preprocessor', preprocessor),
#                       ('classifier', LogisticRegression(solver='lbfgs'))])


# list of (estimator, param_grid), where param_grid is used in GridSearchCV
classifiers = [
    (make_pipeline(
        #preprocessor,
        LogisticRegression(solver='lbfgs', random_state=0)), {
        'logisticregression__C': np.logspace(-2, 1, 2)
    }),
     (make_pipeline(
#         #preprocessor,
#         #LinearSVC(random_state=0)), {
#         #'linearsvc__C': np.logspace(-2, 5, 10)
#     #}https://stackoverflow.com/questions/47312432/attributeerrorlinearsvc-object-has-no-attribute-predict-proba
        SVC(kernel='linear',probability=True)),
        {'svc__C': [1,]}
    ),
#      (make_pipeline(
#         #preprocessor,
#         SGDClassifier(random_state=0, early_stopping=True, loss='modified_huber')), {
#         'sgdclassifier__eta0': np.logspace(-1, 2, 10)
#     }),
    (make_pipeline(
        #preprocessor,
        #KBinsDiscretizer(encode='onehot'),
        PolynomialFeatures(),
        LogisticRegression(solver='lbfgs', random_state=0)), {
            #'kbinsdiscretizer__n_bins': np.arange(2, 3),
            'logisticregression__C': np.logspace(-2, 1, 2),
        }),
    (make_pipeline(
        #preprocessor,
        #KBinsDiscretizer(encode='onehot'), 
        PolynomialFeatures(),
        #LinearSVC(random_state=0)), {
        #    'kbinsdiscretizer__n_bins': np.arange(2, 10),
        #    'linearsvc__C': np.logspace(-2, 2, 10),
        #}
        SVC(kernel='linear',probability=True)),
        #{'svc__C': np.logspace(-1, 2, 2)}
        {'svc__C': [1,]}
    ),
    (make_pipeline(
        #preprocessor,
        GradientBoostingClassifier(n_estimators=50, random_state=0)), {
        'gradientboostingclassifier__learning_rate': np.logspace(-4, 0, 2)
    }),
#     (make_pipeline(
#         #preprocessor,
#         SVC(random_state=0, gamma='scale', probability=True)), {
#         'svc__C': np.logspace(-1, 2, 10)
#     }),
#      (make_pipeline(
#         #preprocessor,
#         AdaBoostClassifier(random_state=0 )), {
#         'adaboostclassifier__learning_rate': np.logspace(-1, 2, 10)
#     }),
     (make_pipeline(
          #preprocessor,
    xgb.XGBClassifier(
                max_depth = 7,
                n_estimators=100,
                learning_rate=0.1, 
                nthread=4,
                subsample=1.0,
                colsample_bytree=0.5,
                min_child_weight = 3,
               # max_delta_step = 7,
                seed=1305)), {
        'xgbclassifier__learning_rate': np.logspace(-2, -1, 2)
    }),
]

names = [get_name(e) for e, g in classifiers]
#train[['Age', 'Fare','SibSp', 'Parch','Sex','Embarked','Pclass']]
#n_samples = 100
datasets = [
    (train[['Age', 'Fare','SibSp', 'Parch','sex_integer','embarked_integer','Pclass']], train[['Survived']]),
   (train[['Age', 'Fare','SibSp', 'Parch','sex_integer','embarked_integer','Pclass','IsCabinNull']], train[['Survived']]),
   (train[['Age', 'Fare','SibSp', 'Parch','sex_integer','embarked_integer','Pclass','ngroup']], train[['Survived']]),
   (train[['age_log', 'fare_log','SibSp', 'Parch','sex_integer','embarked_integer','Pclass']], train[['Survived']]),
   (train[['age_scale', 'fare_scale','SibSp', 'Parch','sex_integer','embarked_integer','Pclass']], train[['Survived']]),
    (train[['age_bins', 'Fare_bins', 'SibSp', 'Parch','sex_integer','embarked_integer','Pclass','SibParchSum','IsWithFamily']], train[['Survived']]),
    (train_after_normalizer, train[['Survived']])
]

fig, axes = plt.subplots(nrows=len(datasets), ncols=len(classifiers) + 1,
                         figsize=(21, 9))

cm = plt.cm.PiYG
cm_bright = ListedColormap(['#b30065', '#178000'])
analysis_results = pd.DataFrame(columns=['Dataset', 
                                         'Classifier', 'Accuracy', 'auc', 
                                         'logloss', 'f1_score','Best params'])
# iterate over datasets
row = 0
for ds_cnt, (X, y) in enumerate(datasets):
    print('\ndataset %d\n---------' % ds_cnt)

    # preprocess dataset, split into training and test part
    #X = StandardScaler().fit_transform(X)
    #X = preprocessor.fit_transform(X)
    X = X.values
    X_train, X_test, y_train, y_test = train_test_split(
        X, y.values.ravel(), test_size=.5, random_state=42, stratify=y)

    # create the grid for background colors
    x_min, x_max = X[:, 0].min() - .5, X[:, 0].max() + .5
    y_min, y_max = X[:, 1].min() - .5, X[:, 1].max() + .5
    xx, yy = np.meshgrid(
        np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))

    # plot the dataset first
    ax = axes[ds_cnt, 0]
    if ds_cnt == 0:
        ax.set_title("Input data")
    # plot the training points
    ax.scatter(X_train[:, 0], X_train[:, 1],
               #c=y_train, cmap=cm_bright,
               edgecolors='k')
    # and testing points
    ax.scatter(X_test[:, 0], X_test[:, 1],
               #c=y_test, cmap=cm_bright, 
               alpha=0.6,
               edgecolors='k')
    ax.set_xlim(xx.min(), xx.max())
    ax.set_ylim(yy.min(), yy.max())
    ax.set_xticks(())
    ax.set_yticks(())

    # iterate over classifiers
    
    for est_idx, (name, (estimator, param_grid)) in \
            enumerate(zip(names, classifiers)):
        ax = axes[ds_cnt, est_idx + 1]

        clf = GridSearchCV(estimator=estimator, param_grid=param_grid, cv=2,
                           iid=False,n_jobs=-1, refit=True)
        with ignore_warnings(category=ConvergenceWarning):
            clf.fit(X_train, y_train)
        score = clf.score(X_test, y_test)
        print('%s: %.2f' % (name, score))
        print('    Best params')
        print(clf.best_params_)
        clf_preds = clf.predict_proba(X_test)[:, 1]
        clf_performance = roc_auc_score(y_test, clf_preds)
        
        print('%s: %.2f' % ('auc', clf_performance))
        clf_log_loss = log_loss(y_test, clf_preds)
        print('%s: %.2f' % ('log loss', clf_log_loss))
        clf_f1_score = f1_score(y_test, clf.predict(X_test))
        print('%s: %.2f' % ('f1 score', clf_f1_score))
        print('')
#         analysis_results = analysis_results.append([str('\ndataset %d\n---------' % ds_cnt), 
#                                  name, score, clf_performance,
#                                  clf_log_loss,clf_f1_score, str(clf.best_params_)], ignore_index=True)
        analysis_results.loc[row, 'Dataset'] = str('dataset%d' % ds_cnt)
        analysis_results.loc[row, 'Classifier'] = name
        analysis_results.loc[row, 'Accuracy'] = score
        analysis_results.loc[row, 'auc'] = clf_performance
        analysis_results.loc[row, 'logloss'] = clf_log_loss
        analysis_results.loc[row, 'f1_score'] = clf_f1_score
        analysis_results.loc[row, 'Best params'] = str(clf.best_params_)
        
        row = row + 1
        # plot the decision boundary. For that, we will assign a color to each
        # point in the mesh [x_min, x_max]*[y_min, y_max].
        
#         if hasattr(clf, "decision_function"):
#             Z = clf.decision_function(X, y)
#         else:
#             Z = clf.predict_proba(X,y)[:, 1]

#         # put the result into a color plot
#         Z = Z.reshape(xx.shape)
#         ax.contourf(xx, yy, Z, 
#                     #cmap=cm, 
#                     alpha=.8)

        # plot the training points
        ax.scatter(X_train[:, 0], X_train[:, 1], 
                   #c=y_train, cmap=cm_bright,
                   edgecolors='k')
        # and testing points
        ax.scatter(X_test[:, 0], X_test[:, 1], 
                   #c=y_test, cmap=cm_bright,
                   edgecolors='k', alpha=0.6)
        ax.set_xlim(xx.min(), xx.max())
        ax.set_ylim(yy.min(), yy.max())
        ax.set_xticks(())
        ax.set_yticks(())

        if ds_cnt == 0:
            ax.set_title(name.replace(' + ', '\n'))
        ax.text(0.95, 0.06, ('%.2f' % score).lstrip('0'), size=15,
                bbox=dict(boxstyle='round', alpha=0.8, facecolor='white'),
                transform=ax.transAxes, horizontalalignment='right')


plt.tight_layout()

# Add suptitles above the figure
plt.subplots_adjust(top=0.90)
suptitles = [
    'Linear classifiers',
    #'Feature discretization and linear classifiers',
    'Feature polynomials and linear classifiers',
    'Non-linear classifiers',
]
for i, suptitle in zip([1, 3, 5], suptitles):
    ax = axes[0, i]
    ax.text(1.05, 1.25, suptitle, transform=ax.transAxes,
            horizontalalignment='center', size='x-large')
plt.show()

In [ ]:
analysis_results

In [ ]:
analysis_results_melt = pd.melt(analysis_results, id_vars=['Dataset', 'Classifier'], value_vars=['Accuracy','auc','logloss','f1_score'])

In [ ]:
g = sns.catplot(x="Dataset", y="Accuracy", hue="Classifier", kind='bar', data=analysis_results, aspect=2)

In [ ]:
#sns.set(rc={'figure.figsize':(15,10)})
g = sns.catplot(x="Dataset", y="value" ,row="variable", data = analysis_results_melt,hue="Classifier",
                 kind="bar", size=4, aspect=2)
  

In [ ]:
grid = sns.FacetGrid(analysis_results_melt, row="variable", hue="Classifier", palette="tab20c", aspect=2)
grid.map(plt.plot, "Dataset", "value", marker="o")
grid.fig.tight_layout(w_pad=1)

In [ ]:
#train[['Age', 'Fare','SibSp', 'Parch','Sex','Embarked','Pclass']].head()

In [ ]:
train.Age.median()

In [ ]:
test.Age.median()

In [ ]:
test.info()

In [ ]:
#https://www.kaggle.com/diegosch/classifier-evaluation-using-confusion-matrix
_ = train.groupby('Pclass')['Fare'].plot(kind='hist', legend=True, alpha=0.8)

In [ ]:
print(train.groupby('Pclass')['Fare'].mean())
print(test.groupby('Pclass')['Fare'].mean())
print(test[test.Fare.isnull()])


In [ ]:
test.loc[test.Fare.isnull(),'Fare'] = 13

In [ ]:
test.loc[test.PassengerId==1044]

In [ ]:
#https://www.kaggle.com/diegosch/classifier-evaluation-using-confusion-matrix
_=train.groupby(['Pclass','Sex'])['Age'].plot(kind='hist', legend=True, alpha=0.8)

In [ ]:
#https://www.kaggle.com/diegosch/classifier-evaluation-using-confusion-matrix
_=train.groupby(['SibSp'])['Age'].plot(kind='hist', legend=True, alpha=0.8)

In [ ]:
#https://www.kaggle.com/diegosch/classifier-evaluation-using-confusion-matrix
_=train.groupby(['SibSp','Sex'])['Age'].plot(kind='hist', legend=True, alpha=0.8)

In [ ]:
#https://www.kaggle.com/diegosch/classifier-evaluation-using-confusion-matrix
_=train.groupby(['Parch'])['Age'].plot(kind='hist', legend=True, alpha=0.8)

In [ ]:
#https://www.kaggle.com/diegosch/classifier-evaluation-using-confusion-matrix
_=train.groupby(['Parch','Sex'])['Age'].plot(kind='hist', legend=True, alpha=0.8)

In [ ]:
test.groupby(['Pclass','Sex'])['Age'].mean()

In [ ]:
#train.loc[train['Age'].isnull() & (train.Pclass==1) &  (train.Sex=='female'), 'Age']=35

In [ ]:
#train.loc[train['Age'].isnull() & (train.Pclass==1) &  (train.Sex=='male'), 'Age']=41

In [ ]:
#train.loc[train['Age'].isnull() & (train.Pclass==1) , 'Age']=

In [ ]:
sns.countplot(x='Survived', hue='Sex', data=train)

In [ ]:
sns.countplot(x='Survived', hue='Embarked', data=train)

In [ ]:
sns.countplot(x='Survived', hue='Pclass', data=train)

In [ ]:
sns.catplot(x='Survived', hue='Sex', col = 'Pclass', kind='count',data=train)

In [ ]:
#https://www.kaggle.com/singh001avinash/nyc-flight-data-eda
#https://seaborn.pydata.org/generated/seaborn.catplot.html
sns.catplot(x="Sex", y="Fare", hue="Survived", data=train)

In [ ]:
sns.catplot(x="Survived", y="Age", hue="Sex", data=train, kind='box')

In [ ]:
#sns.catplot(x="Sex", y="Age", hue="Survived", col="Pclass", data=train)

In [ ]:
sns.countplot(train.Sex)

In [ ]:
sns.countplot(test.Sex)

In [ ]:
#sns.distplot(train.Fare)

In [ ]:
#https://www.kaggle.com/tanetboss/starter-guide-preprocessing-randomforest
train['Survived'].value_counts(sort = False)

In [ ]:
#https://www.kaggle.com/tanetboss/starter-guide-preprocessing-randomforest
#missing data
total = train.isnull().sum().sort_values(ascending=False)
percent = (train.isnull().sum()/train.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head(5)

In [ ]:
total = test.isnull().sum().sort_values(ascending=False)
percent = (test.isnull().sum()/test.isnull().count()).sort_values(ascending=False)
missing_test_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_test_data.head(5)

In [ ]:
#https://stackoverflow.com/questions/30503321/finding-count-of-distinct-elements-in-dataframe-in-each-column
train.nunique()

In [ ]:
y_train = train.Survived

In [ ]:
#train = train.drop('Name', axis=1)
#train = train.drop('Ticket', axis=1)
#train = train.drop('PassengerId', axis=1)

In [ ]:
train.groupby('Survived').mean()

In [ ]:
train.groupby(train.Cabin.isnull(), as_index=False).size()

In [ ]:
train[['Pclass','Age','Cabin']].head()

In [ ]:
#train[['Pclass','Age']].groupby(train.Cabin.isnull(), as_index=False).expanding().mean()

In [ ]:
#train.groupby(['Pclass','Embarked','Sex', 'SibSp', 'Parch']).ngroup()

In [ ]:
train.groupby(train.Cabin.isnull()).mean().iloc[0]

In [ ]:
train.groupby(train.Cabin.isnull()).mean().iloc[1]

In [ ]:
#train.groupby(train.Cabin.isnull()).get_group(True)

In [ ]:
#train.groupby(train.Cabin.isnull()).get_group(False)

In [ ]:
train.groupby('Pclass').mean()

In [ ]:
train.groupby('Embarked').mean()

In [ ]:
train.groupby('Sex').mean()

In [ ]:
train.corr()

In [ ]:
#train.groupby('Pclass').corr()

In [ ]:
train.groupby('SibSp').corr()

In [ ]:
train.groupby('SibSp').mean()

In [ ]:
train.groupby('Parch').corr()

In [ ]:
train.groupby('Parch').mean()

In [ ]:
train['SibSp'].value_counts(sort=False)

In [ ]:
train[['SibSp', 'Survived']].groupby('SibSp')['Survived'].sum()

In [ ]:
(train[['SibSp', 'Survived']].groupby('SibSp')['Survived'].sum()/train['SibSp'].value_counts(sort=False))*100

In [ ]:
train.Parch.value_counts(sort=False)

In [ ]:
train[['Parch', 'Survived']].groupby('Parch')['Survived'].sum()

In [ ]:
(train[['Parch', 'Survived']].groupby('Parch')['Survived'].sum()/train.Parch.value_counts(sort=False))*100

In [ ]:
(train[['Pclass', 'Survived']].groupby('Pclass')['Survived'].sum()/train.Pclass.value_counts(sort=False))*100

In [ ]:
#train.groupby('Embarked').corr()

In [ ]:
#train = train.drop('Survived', axis=1)

In [ ]:
#https://www.kaggle.com/kikexclusive/curiosity-didn-t-kill-the-cat-all-in-one
from IPython.display import display, HTML
display(train.describe(include="all").T)

In [ ]:
#https://stackoverflow.com/questions/27424178/faster-way-to-remove-outliers-by-group-in-large-pandas-dataframe
#https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.stats.mstats.winsorize.html
#https://subscription.packtpub.com/book/big_data_and_business_intelligence/9781785282287/4/ch04lvl1sec54/winsorizing-data
#https://towardsdatascience.com/ways-to-detect-and-remove-the-outliers-404d16608dba
from scipy.stats import mstats

#train['Fare'] = mstats.winsorize(train['Fare'], limits = (0.05,0.25))

In [ ]:
#train['Fare'].describe()

In [ ]:
display(test.describe(include="all").T)

In [ ]:
#test['Fare'] = mstats.winsorize(test['Fare'], limits = (0.05,0.25))

In [ ]:
#test['Fare'].describe()

In [ ]:
train.info()

In [ ]:
train.Embarked.value_counts()

In [ ]:
#https://www.kaggle.com/sudalairajkumar/simple-exploration-baseline-ga-customer-revenue
const_cols = [c for c in train.columns if train[c].nunique(dropna=False)==1 ]
const_cols

In [ ]:
#https://www.kaggle.com/sudalairajkumar/simple-exploration-baseline-ga-customer-revenue
train.groupby('Pclass')['Age'].agg(['size', 'count', 'mean'])

In [ ]:
train.describe()

In [ ]:
#https://www.kaggle.com/sudalairajkumar/simple-exploration-baseline-ga-customer-revenue
train.groupby('Sex')['Age'].agg(['size', 'count', 'mean'])

In [ ]:
#https://www.kaggle.com/sudalairajkumar/simple-exploration-baseline-ga-customer-revenue
train.groupby('Embarked')['Age'].agg(['size', 'count', 'mean'])

In [ ]:
#https://scikit-learn.org/stable/auto_examples/compose/plot_column_transformer_mixed_types.html#sphx-glr-auto-examples-compose-plot-column-transformer-mixed-types-py
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV

np.random.seed(2019)


numeric_features = ['Age', 'Fare']
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])

categorical_features = ['Embarked', 'Sex', 'Pclass']
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])


clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', LogisticRegression(solver='lbfgs'))])


Xt_train, Xt_test, yt_train, yt_test = train_test_split(train, y_train,stratify=y_train, test_size=0.3)

clf.fit(Xt_train, yt_train)
print("model score: %.3f" % clf.score(Xt_test, yt_test))
print(cross_val_score(clf, train, y_train, cv=5))

In [ ]:
# from sklearn.compose import ColumnTransformer
# from sklearn.pipeline import Pipeline
# from sklearn.impute import SimpleImputer
# from sklearn.preprocessing import StandardScaler, OneHotEncoder
# from sklearn.linear_model import LogisticRegression


# np.random.seed(2019)


# #numeric_features = ['Age', 'Fare','SibSp','Parch']
# numeric_features = ['Age', 'Fare']
# numeric_transformer = Pipeline(steps=[
#     ('imputer', SimpleImputer(strategy='median')),
#     ('scaler', StandardScaler())])

# categorical_features = ['Embarked', 'Sex', 'Pclass']
# categorical_transformer = Pipeline(steps=[
#     ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
#     ('onehot', OneHotEncoder(handle_unknown='ignore'))])

# preprocessor = ColumnTransformer(
#     transformers=[
#         ('num', numeric_transformer, numeric_features),
#         ('cat', categorical_transformer, categorical_features)])

# from xgboost import XGBClassifier
# gbm = XGBClassifier(
#     max_depth=3, n_estimators=300, learning_rate=0.05
# ).fit(preprocessor.fit_transform(train), y_train)

# #clf = Pipeline(steps=[('preprocessor', preprocessor),
# #                      ('classifier', LogisticRegression(solver='lbfgs'))])

# import eli5
# eli5.show_weights(gbm, top=13)
# print(cross_val_score(gbm, preprocessor.fit_transform(train), y_train,cv=5 ) )

In [ ]:
#train['SibParch'] = train['SibSp']*train['Parch'] + train['SibSp'] + train['Parch'] 
#test['SibParch'] = test['SibSp']*test['Parch'] + test['SibSp'] + test['Parch'] 

In [ ]:
#sns.pairplot(train[['Age', 'Fare','SibSp', 'Parch','Sex','Embarked','Pclass', 'Survived']], hue='Survived')

In [ ]:
#features= ['Age','SibSp','Parch', 'Fare', 'Pclass','Sex','Cabin','Embarked']
features= ['Age', 'Fare','SibSp', 'Parch','Sex','Embarked','Pclass']
#num_features = ['Age','SibSp','Parch', 'Fare']
num_features = ['SibSp','Parch']
num_features_to_scale = ['Age', 'Fare']
cat_features = ['Sex', 'Embarked','Pclass']
train_test_concat = pd.concat([train[features],test[features]])

In [ ]:
train_test_concat.info()

In [ ]:
#https://www.kaggle.com/moghazy/eda-for-iris-dataset-with-svm-and-pca
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(train_test_concat[num_features_to_scale])

In [ ]:

train_scaled = train[num_features_to_scale].copy()
train_scaled = scaler.transform(train_scaled)
train_scaled = pd.DataFrame(train_scaled,columns=num_features_to_scale)
train_scaled = pd.concat([train_scaled,train[cat_features],train[num_features]],axis=1)
print(train_scaled.info())

In [ ]:

test_scaled = test[num_features_to_scale].copy()
test_scaled = scaler.transform(test_scaled)
test_scaled = pd.DataFrame(test_scaled,columns=num_features_to_scale)
test_scaled = pd.concat([test_scaled,test[cat_features],test[num_features]],axis=1)
print(test_scaled.info())

In [ ]:
train_scaled.describe()

In [ ]:
test_scaled.describe()

In [ ]:
#https://stackoverflow.com/questions/53254292/pandas-simpleimputer-preserve-datatypes

from sklearn.impute import SimpleImputer
my_imputer = SimpleImputer(strategy='most_frequent')
my_imputer.fit(pd.concat([train_scaled,test_scaled]))
data_with_imputed_values = pd.DataFrame(
     my_imputer.transform(train_scaled), columns=train_scaled.columns
).astype(train_scaled.dtypes.to_dict())

test_data_with_imputed_values = pd.DataFrame(
     my_imputer.transform(test_scaled), columns=test_scaled.columns
).astype(test_scaled.dtypes.to_dict())

print(data_with_imputed_values.info())
print(test_data_with_imputed_values.info())

In [ ]:
#https://markhneedham.com/blog/2017/07/05/pandasscikit-learn-get_dummies-testtrain-sets-valueerror-shapes-not-aligned/
all_data = pd.concat([data_with_imputed_values,test_data_with_imputed_values])
#https://stackoverflow.com/questions/47537823/futurewarning-specifying-categories-or-ordered-in-astype-is-deprecated

for column in all_data.select_dtypes(include=[np.object]).columns:
    data_with_imputed_values[column] = pd.Categorical(data_with_imputed_values[column], categories = all_data[column].unique())
    test_data_with_imputed_values[column] = pd.Categorical(test_data_with_imputed_values[column], categories = all_data[column].unique())


In [ ]:
#data_with_imputed_values.info()

In [ ]:
#test_data_with_imputed_values.info()

In [ ]:
data_with_imputed_values = pd.get_dummies(data_with_imputed_values,drop_first=True)
test_data_with_imputed_values = pd.get_dummies(test_data_with_imputed_values,drop_first=True)

In [ ]:
data_with_imputed_values.info()

In [ ]:
test_data_with_imputed_values.info()

In [ ]:
#https://www.kaggle.com/tanetboss/starter-guide-preprocessing-randomforest
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV


param_grid  = { 
                'n_estimators' : [500,1200],
               # 'min_samples_split': [2,5,10,15,100],
               # 'min_samples_leaf': [1,2,5,10],
                'max_depth': range(1,5,2),
                'max_features' : ('log2', 'sqrt'),
                'class_weight':[{1: w} for w in [1,1.5]]
              }

GridRF = GridSearchCV(RandomForestClassifier(random_state=15), param_grid)

GridRF.fit(data_with_imputed_values, y_train)
#RF_preds = GridRF.predict_proba(X_test)[:, 1]
#RF_performance = roc_auc_score(Y_test, RF_preds)

print(
    #'DecisionTree: Area under the ROC curve = {}'.format(RF_performance)
     "\nBest parameters \n" + str(GridRF.best_params_))


rf = RandomForestClassifier(random_state=15,**GridRF.best_params_)
rf.fit(data_with_imputed_values, y_train)

Rfclf_fea = pd.DataFrame(rf.feature_importances_)
#print(Rfclf_fea)
Rfclf_fea["Feature"] = list(data_with_imputed_values.columns) 
Rfclf_fea.sort_values(by=0, ascending=False).head(10)

In [ ]:
cross_val_score(rf, data_with_imputed_values, y_train,cv=5 ) 

In [ ]:
#tn, fp, fn, tp = confusion_matrix(train_copy['Survived'], train_copy['Prediction']).ravel()
#(tn, fp, fn, tp)

In [ ]:
#https://www.kaggle.com/moghazy/eda-for-iris-dataset-with-svm-and-pca
# import seaborn as sns
# import matplotlib.pyplot as plt

# df_imp = pd.concat([data_with_imputed_values, pd.DataFrame(y_train)], axis=1)
# print(df_imp.columns)
# corr = df_imp.iloc[:,[0,3,4,5,6,157]].corr()
# f, ax = plt.subplots(figsize=(15, 10))
# cmap = sns.diverging_palette(220, 10, as_cmap=True)
# sns.heatmap(corr, cmap=cmap, vmax=.3, center=0,
#             square=True, linewidths=.5)

In [ ]:
print(type(data_with_imputed_values))
print(data_with_imputed_values.shape)

In [ ]:
one = OneHotEncoder(handle_unknown='ignore')
simple = SimpleImputer(strategy='constant', fill_value='missing')
pre = one.fit_transform(simple.fit_transform(train[['Embarked', 'Sex', 'Pclass']]))

In [ ]:
one.categories_

In [ ]:
pre.shape

In [ ]:
pre.toarray()

In [ ]:
one.get_feature_names()

In [ ]:
data_with_imputed_values.columns

https://www.kaggle.com/arthurtok/introduction-to-ensembling-stacking-in-python

In [ ]:
#data_with_imputed_values.describe()

In [ ]:
#test_data_with_imputed_values.describe()

In [ ]:
#https://stackoverflow.com/questions/19071199/pandas-dataframe-drop-columns-whose-name-contains-a-specific-string
def drop_cols_with_str(df,str_v):
    return df[df.columns.drop(list(df.filter(regex=str_v)))]
data_with_imputed_values = drop_cols_with_str(data_with_imputed_values,'Cabin_')  
#data_with_imputed_values = data_with_imputed_values.drop('Sex_female', axis=1)
#data_with_imputed_values = data_with_imputed_values.drop('Embarked_Q', axis=1)
test_data_with_imputed_values = drop_cols_with_str(test_data_with_imputed_values,'Cabin_')  
#test_data_with_imputed_values = test_data_with_imputed_values.drop('Sex_female', axis=1)
#test_data_with_imputed_values = test_data_with_imputed_values.drop('Embarked_Q', axis=1)

In [ ]:
#data_with_imputed_values = data_with_imputed_values[data_with_imputed_values.columns.drop('Fare')]


In [ ]:
#test_data_with_imputed_values = test_data_with_imputed_values[test_data_with_imputed_values.columns.drop('Fare')]

In [ ]:
#data_with_imputed_values.describe()

In [ ]:
#test_data_with_imputed_values.describe()

https://www.kaggle.com/tejasrinivas/xgb-baseline-comments-classification

In [ ]:

from sklearn.model_selection import train_test_split
import xgboost as xgb

#data_with_imputed_values_wo_Fare = data_with_imputed_values.drop('Fare', axis=1)
X_t, X_v, y_t, y_v = train_test_split(data_with_imputed_values,y_train, stratify=y_train, test_size=0.2, random_state=2019)
#X_t, X_v, y_t, y_v = train_test_split(data_with_imputed_values_wo_Fare,y_train, test_size=0.2, random_state=2019)


def runXGB(X_t, X_v, y_t, y_v, feature_names=None, seed_val=2017, num_rounds=200):
    param = {}
    param['objective'] = 'binary:logistic'
    param['eta'] = 0.1
    param['max_depth'] = 6
    param['silent'] = 1
    param['eval_metric'] = 'auc'
    param['min_child_weight'] = 1 #3
    param['subsample'] = 0.5
   # param['colsample_bytree'] = 0.5
    param['seed'] = seed_val
   # param['max_delta_step'] = 8
#     param['objective'] = 'binary:logistic'
#     param['eta'] = 0.1
#     param['max_depth'] = 6
#     param['silent'] = 1
#     param['eval_metric'] = 'auc'
#     param['min_child_weight'] = 1
#     param['subsample'] = 0.5
#     param['colsample_bytree'] = 0.5
#     param['seed'] = seed_val
    
    
    num_rounds = num_rounds

    plst = list(param.items())
    xgtrain = xgb.DMatrix(X_t, label=y_t)


    xgtest = xgb.DMatrix(X_v, label=y_v)
    watchlist = [ (xgtrain,'train'), (xgtest, 'test') ]
    model = xgb.train(plst, xgtrain, num_rounds, watchlist, early_stopping_rounds=20)
    #model = xgb.train(plst, xgtrain, num_rounds, watchlist)
    return model    

model = runXGB(X_t, X_v, y_t, y_v)


In [ ]:
preds = model.predict(xgb.DMatrix(data_with_imputed_values))

In [ ]:
from sklearn.metrics import confusion_matrix
pred_t = model.predict(xgb.DMatrix(data_with_imputed_values))
train_copy['Prediction'] = (pred_t > 0.4).astype(int)
tn, fp, fn, tp = confusion_matrix(train_copy['Survived'], train_copy['Prediction']).ravel()
(tn, fp, fn, tp)

In [ ]:
xgb.plot_importance(model)

In [ ]:
# print('Predict the probabilities based on features in the test set')
# pred = model.predict(xgb.DMatrix(test_data_with_imputed_values))
# #pred = clf.predict(test_data_with_imputed_values)
# out_df = pd.DataFrame({"PassengerId":test["PassengerId"].values})
# out_df['Survived'] = (pred>0.5).astype(int)

# out_df.to_csv("submission.csv", index=False)
# print('out_df.to_csv')
# print(out_df.head())

In [ ]:
def runXGBcv(X, y, feature_names=None, seed_val=2017, num_rounds=100):
    param = {}
    param['objective'] = 'binary:logistic'
    param['eta'] = 0.05
    param['max_depth'] = 6
    param['silent'] = 1
    param['eval_metric'] = 'auc'
    param['min_child_weight'] = 3
    param['subsample'] = 0.5
    param['colsample_bytree'] = 0.5
    param['seed'] = seed_val
  #  param['max_delta_step'] = 8
#     param['objective'] = 'binary:logistic'
#     param['eta'] = 0.1
#     param['max_depth'] = 6
#     param['silent'] = 1
#     param['eval_metric'] = 'auc'
#     param['min_child_weight'] = 1
#     param['subsample'] = 0.5
#     param['colsample_bytree'] = 0.5
#     param['seed'] = seed_val
    
    num_rounds = num_rounds

    plst = list(param.items())
    xgtrain = xgb.DMatrix(X, label=y)

    result = xgb.cv(plst, xgtrain, num_rounds, nfold=5,stratified=True, 
           folds=None, metrics=(), obj=None, feval=None, maximize=False, 
           early_stopping_rounds=20, fpreproc=None, as_pandas=True, 
           verbose_eval=None, show_stdv=True, seed=2019, callbacks=None, shuffle=True)
    return result    

result = runXGBcv(data_with_imputed_values, y_train)
#result

In [ ]:
data_with_imputed_values.describe()

https://www.kaggle.com/cast42/xg-cv

In [ ]:
from sklearn.metrics import roc_auc_score
clf = xgb.XGBClassifier(
                max_depth = 7,
                n_estimators=700,
                learning_rate=0.1, 
                nthread=4,
                subsample=1.0,
                colsample_bytree=0.5,
                min_child_weight = 3,
               # max_delta_step = 7,
                seed=1301)

#                 max_depth = 7,
#                 n_estimators=700,
#                 learning_rate=0.1,  
#                 nthread=4,
#                 subsample=0.4, #1.0
#                 colsample_bytree=0.4,
#                 min_child_weight = 5,
#                 seed=1301)
                
                
xgb_param = clf.get_xgb_params()

cvresult = xgb.cv(xgb_param, xgb.DMatrix(data_with_imputed_values,y_train), 
                  num_boost_round=400, nfold=15, metrics=['auc'],
                  early_stopping_rounds=30, 
                  stratified=True,
                  as_pandas = True, show_stdv= True, seed=1301)
print('Best number of trees = {}'.format(cvresult.shape[0]))
clf.set_params(n_estimators=cvresult.shape[0])

clf.fit(data_with_imputed_values, y_train, eval_metric='auc')
print('Overall AUC:', roc_auc_score(y_train, clf.predict_proba(data_with_imputed_values)[:,1]))
# print('Predict the probabilities based on features in the train set')
# pred = clf.predict_proba(data_with_imputed_values, ntree_limit=cvresult.shape[0])

#pred = clf.predict(test_data_with_imputed_values)
#pred = clf.predict(test_data_with_imputed_values)


In [ ]:
#cvresult

In [ ]:
clf.feature_importances_

In [ ]:
xgb.plot_importance(clf)

In [ ]:
#cv_score = cross_val_score(clf, data_with_imputed_values, y_train,cv=10 ) 

In [ ]:
#cv_score

In [ ]:
#cv_score.std()

In [ ]:
print('Predict the probabilities based on features in the test set')
pred_XGBC = clf.predict(test_data_with_imputed_values)
#pred = clf.predict(test_data_with_imputed_values)
out_df = pd.DataFrame({"PassengerId":test["PassengerId"].values})
out_df['Survived'] = pred_XGBC

out_df.to_csv("submission.csv", index=False)
print('out_df.to_csv')
print(out_df.head())

In [ ]:
#http://danielhnyk.cz/how-to-use-xgboost-in-python/
# import scipy.stats as st

# one_to_left = st.beta(10, 1)  
# from_zero_positive = st.expon(0, 50)

# params = {  
#     "n_estimators": st.randint(3, 40),
#     "max_depth": st.randint(3, 40),
#     "learning_rate": st.uniform(0.05, 0.4),
#     "colsample_bytree": one_to_left,
#     "subsample": one_to_left,
#     "gamma": st.uniform(0, 10),
#     'reg_alpha': from_zero_positive,
#     "min_child_weight": from_zero_positive,
# }

# xgbreg = xgb.XGBClassifier(nthreads=-1) 
# from sklearn.model_selection import RandomizedSearchCV

# random_search = RandomizedSearchCV(xgbreg, params, n_jobs=1,refit=True,
#                                    n_iter=50,return_train_score=True,verbose=1, cv=5)  
# random_search.fit(data_with_imputed_values, y_train)  
# pred = random_search.predict(test_data_with_imputed_values)

In [ ]:
#https://github.com/dmlc/xgboost/blob/master/demo/guide-python/sklearn_examples.py
# from sklearn.model_selection import GridSearchCV
# search = GridSearchCV(clf,
#                    {'max_depth': [6,7],      #[2,4,5,6,7]
#                     'n_estimators': [100,200,300], 
#                     'learning_rate': [0.08, 0.1],
#                     'min_child_weight':[2,3,4],
#                     'subsample':[0.5,], #[0.1,0.3,0.4,0.5]
#                     'colsample_bytree':[0.4,], #[0.1,0.3,0.4,0.5]
#                     'gamma':[0,0.1,0.2]},
#                    verbose=1,refit=True)
# search.fit(data_with_imputed_values, y_train)
# print(search.best_score_)
# print(search.best_params_)

In [ ]:
# clf = search.best_estimator_

# clf.fit(data_with_imputed_values, y_train, eval_metric='auc')
# print('Overall AUC:', roc_auc_score(y_train, clf.predict_proba(data_with_imputed_values)[:,1]))
#print('Predict the probabilities based on features in the train set')
#pred = clf.predict_proba(data_with_imputed_values, ntree_limit=cvresult.shape[0])

In [ ]:
#https://datascience.stackexchange.com/questions/19882/xgboost-how-to-use-feature-importances-with-xgbregressor
clf.get_booster().get_score(importance_type='weight')

In [ ]:
cross_val_score(clf, data_with_imputed_values, y_train,cv=10 ) 

In [ ]:
# print('Predict the probabilities based on features in the test set')
# pred = clf.predict(test_data_with_imputed_values, ntree_limit=cvresult.shape[0])
# #pred = clf.predict(test_data_with_imputed_values)
# out_df = pd.DataFrame({"PassengerId":test["PassengerId"].values})
# out_df['Survived'] = pred

# out_df.to_csv("submission.csv", index=False)
# print('out_df.to_csv')
# print(out_df.head())

In [ ]:
xgb.plot_importance(clf)

In [ ]:
#clf.get_booster().get_split_value_histogram('Age',as_pandas=True)

In [ ]:
from sklearn.metrics import confusion_matrix
pred_t = clf.predict(data_with_imputed_values, ntree_limit=cvresult.shape[0])
train_copy['Prediction'] = pred_t
tn, fp, fn, tp = confusion_matrix(train_copy['Survived'], train_copy['Prediction']).ravel()
(tn, fp, fn, tp)

In [ ]:
preds_proba = clf.predict_proba(data_with_imputed_values, ntree_limit=cvresult.shape[0])
train_copy['Prediction'] = preds_proba[:,1]

In [ ]:
preds_proba

In [ ]:
train_copy.columns

In [ ]:
train_copy.Pclass.value_counts()

In [ ]:
sns.swarmplot(x="Pclass", y="Prediction", hue="Survived",
               data=train_copy)
#https://stackoverflow.com/questions/30490740/move-legend-outside-figure-in-seaborn-tsplot
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)

In [ ]:
sns.swarmplot(x="Embarked", y="Prediction", hue="Survived",
               data=train_copy)
#https://stackoverflow.com/questions/30490740/move-legend-outside-figure-in-seaborn-tsplot
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)

In [ ]:
#https://seaborn.pydata.org/examples/scatter_bubbles.html

sns.set(style="white")


sns.relplot(x="Age", y="Prediction", hue="Survived", style="Pclass",
            sizes=(40, 400), alpha=.5, palette="muted",
            height=6, data=train_copy)

In [ ]:
# g = sns.lmplot(x="Age", y="Prediction", hue="Sex",
#                truncate=True, height=5, data=train_copy)

In [ ]:
g = sns.lmplot(x="Fare", y="Prediction", hue="Survived",
               truncate=True, height=5, data=train_copy)

In [ ]:
g = sns.lmplot(x="Age", y="Prediction", hue="Survived",
               truncate=True, height=5, data=train_copy)

In [ ]:
g = sns.catplot(x="Survived", y="Prediction", hue="Pclass",
               kind='box', height=5, data=train_copy)

In [ ]:
g = sns.catplot(x="Survived", y="Prediction", hue="Sex",
               kind='box', height=5, data=train_copy)

In [ ]:
g = sns.catplot(x="Survived", y="Prediction", hue="Embarked",
               kind='box', height=5, data=train_copy)

In [ ]:
 sns.relplot(x="Age", y="Prediction", hue="Survived", data=train_copy)

In [ ]:
sns.relplot(x="Age", y="Prediction", hue="Survived", col="Sex", data=train_copy)


In [ ]:
#sns.relplot(x="Age", y="Prediction", hue="Embarked",style='Survived', col="Sex", row='Pclass',data=train_copy)
sns.relplot(x="Age", y="Prediction", col="Sex",hue='Survived', row='Pclass',data=train_copy)


In [ ]:
sns.relplot(x="Fare", y="Prediction", hue="Survived", col="Sex", row='Pclass', data=train_copy)


In [ ]:
#https://stackoverflow.com/questions/46045750/python-distplot-with-multiple-distributions
target_0 = train_copy.loc[train_copy['Survived'] == 0]
target_1 = train_copy.loc[train_copy['Survived'] == 1]


_= sns.distplot(target_0[['Prediction']], hist=False, rug=True)
_= sns.distplot(target_1[['Prediction']], hist=False, rug=True)



In [ ]:
#ax = sns.catplot(x="Survived", y="Prediction",col='Pclass', data=train_copy, estimator=median,height=4, aspect=.7,kind='bar')

In [ ]:
#ax = sns.catplot(x="Survived", y="Prediction",col='Embarked', data=train_copy, estimator=median,height=4, aspect=.7,kind='bar')

In [ ]:
#https://seaborn.pydata.org/generated/seaborn.barplot.html
#https://www.kaggle.com/nadintamer/titanic-survival-predictions-beginner
ax = sns.catplot(x="Survived", y="Prediction",col='SibSp', data=train_copy, estimator=median,height=4, aspect=.7,kind='bar')

In [ ]:
((train[['SibSp', 'Survived']].groupby('SibSp')['Survived'].sum()/train['SibSp'].value_counts(sort=False))*100).plot(kind='bar')

In [ ]:
ax = sns.catplot(x="Survived", y="Prediction",col='Parch', data=train_copy, estimator=median,height=4, aspect=.7,kind='bar')

In [ ]:
((train[['Parch', 'Survived']].groupby('Parch')['Survived'].sum()/train['Parch'].value_counts(sort=False))*100).plot(kind='bar')

In [ ]:
#https://python-graph-gallery.com/111-custom-correlogram/
#_= sns.pairplot(data_with_imputed_values, kind="scatter", hue="Survived", plot_kws=dict(alpha= 0.5))


In [ ]:
temp = train_copy.copy()
temp['Family_members'] = temp.SibSp + temp.Parch
sns.catplot(x="Family_members", y="Prediction",kind='box', hue="Survived", col="Sex", row='Pclass', data=temp)


In [ ]:
sns.catplot(x="Embarked", y="Prediction",kind='box', col="Sex",hue='Survived', row='Pclass',data=train_copy)

In [ ]:
#https://github.com/dmlc/xgboost/issues/1725
#https://machinelearningmastery.com/visualize-gradient-boosting-decision-trees-xgboost-python/
# import matplotlib
# xgb.plot_tree(clf, rankdir='LR')
# fig = matplotlib.pyplot.gcf()
# fig.set_size_inches(150, 200)
# fig.savefig('tree.png')